In [1]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from utils.dataset import PandasDataset, RGB2LABTransform
from utils.models import EfficientNet
from utils.metrics import evaluation, format_metrics
import albumentations

In [2]:
output_dimensions = 5
data_dir = '../../../dataset'
ROOT_DIR = "../../"
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: f'{ROOT_DIR}/efficientnet-b0-08094119.pth'
}

In [3]:
print("Cuda", device)

Cuda cuda


In [4]:
val_transform = albumentations.Compose([
    RGB2LABTransform(p=1.0)
])

In [5]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [6]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        f"{ROOT_DIR}/models/with-noise-lab.pth",
        weights_only=True
    )
)

response_0 = evaluation(model, dataloader, device)
result = format_metrics(response_0[0])
print(result)

Loaded pretrained weights for efficientnet-b0


100%|██████████| 796/796 [06:53<00:00,  1.93it/s]


VAL_ACC      Mean: 60.76 | Std: 1.23 | 95% CI: [58.61, 62.81]
VAL_KAPPA    Mean: 0.86 | Std: 0.01 | 95% CI: [0.84, 0.87]
VAL_F1       Mean: 0.55 | Std: 0.01 | 95% CI: [0.53, 0.57]
VAL_RECALL   Mean: 0.55 | Std: 0.01 | 95% CI: [0.53, 0.57]
VAL_PRECISION Mean: 0.57 | Std: 0.01 | 95% CI: [0.55, 0.59]
